In [4]:
# get local path directory
localpath = pathlib.Path.cwd()
# adding Scripts/2023 to system path to find utils.py
t = sys.path.insert(0, localpath.parent / '2023')

In [7]:
localpath

WindowsPath('c:/Users/mbindl/Documents/GitHub/ThresholdEvaluation/Scripts/Vegetation')

In [8]:
import pathlib
import pandas as pd
import sys
# get local path directory
localpath = pathlib.Path.cwd()
# adding Scripts/2023 to system path to find utils.py
sys.path.insert(0, localpath.parent / '2023')
from utils import *

ModuleNotFoundError: No module named 'utils'

In [14]:
# get 2010 Ecobject data
def get_ecobject_2010_data():
    # make sql database connection with pyodbc
    engine = get_conn('sde_tabular')
    # get dataframe from BMP SQL Database
    with engine.begin() as conn:
        # create dataframe from sql query
        df = pd.read_sql("SELECT * FROM sde_tabular.SDE.Vegetation_EcObject_2010", conn)
    return df

# get identity data of 2010 Ecobject data overlayed with Caldor Burn Severity
def get_ecobject_2010_identity_data():
    # make sql database connection with pyodbc
    engine = get_conn('sde_tabular')
    # get dataframe from BMP SQL Database
    with engine.begin() as conn:
        # create dataframe from sql query
        df = pd.read_sql("SELECT * FROM sde_tabular.SDE.ThresholdEvaluation_Vegetation_ID_Ecobejct2010_CaldorVegBurnSeverity", conn)
    return df

In [12]:
df = get_ecobject_2010_data()

In [15]:
dfNew = get_ecobject_2010_identity_data()

In [ ]:
columns_to_keep = [ 'SpatialVar', 'Ownership', 'Development',
                   'WHRTYPE', 'Acres','QMD','Elev_Ft', 'SeralStage', 
                   'TRPA_VegType','gridcode','FID_CaldorBurnSeverity']

dfCal = dfNew[columns_to_keep]
# filter out Development = 1
dfCal = dfCal[dfCal['Development'] =='Undeveloped']
# pivot table to get the sum of Acres for TRPA_VegType, gridcode, SeralStage
dfVegSeverity = dfCal.pivot_table(index=['TRPA_VegType'], columns=['gridcode'], values='Acres', aggfunc='sum').reset_index()
# round acres to 2 decimal places
dfVegSeverity = dfVegSeverity.round(1)
dfVegSeverity

In [39]:
# sum Columns 3 and 4
dfVegSeverity['Total_Burned'] = dfVegSeverity[3] + dfVegSeverity[4]
# rename columns 
dfVegSeverity.rename(columns={0:'Unburned',3:'Moderate', 4:'High'}, inplace=True)

In [ ]:
# filter out Development
df = df[df['Development'] =='Undeveloped']
# pivot Acres in df to get the sum of Acres for TRPA_VegType, gridcode, SeralStage
dfVeg = df.pivot_table(index=['TRPA_VegType'], values='Acres', aggfunc='sum').reset_index()
dfVeg = dfVeg.round(1)
dfVeg

In [ ]:
dfNewVegSummary = pd.merge(dfVeg, dfVegSeverity, on='TRPA_VegType', how='left')
dfNewVegSummary